In [5]:
import tensorflow as tf
import keras
import numpy
import matplotlib
import sklearn
import cv2

In [6]:
train_directory = "/content/drive/MyDrive/ML chest xray data/TRAIN"
val_directory = "/content/drive/MyDrive/ML chest xray data/VALIDATION"
test_directory = "/content/drive/MyDrive/ML chest xray data/TEST"

In [7]:
image_size = (224,224)
batch_size = 32

train_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_directory,
    image_size = image_size,
    batch_size = batch_size,
    label_mode = 'categorical'
)
validation_data = tf.keras.preprocessing.image_dataset_from_directory(
    val_directory,
    image_size = image_size,
    batch_size = batch_size,
    label_mode = 'categorical'
)
test_data = tf.keras.preprocessing.image_dataset_from_directory(
    test_directory,
    image_size = image_size,
    batch_size = batch_size,
    label_mode = 'categorical'
)

Found 6326 files belonging to 4 classes.
Found 38 files belonging to 4 classes.
Found 771 files belonging to 4 classes.


In [8]:
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horozontal"),
    layers.RandomRotation(0.4),
    layers.RandomZoom(0.4),
    layers.RandomTranslation(0.2, 0.2),
    layers.RandomContrast(0.2),
])

In [9]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam

resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in resnet_base.layers:
    layer.trainable = False

model = Sequential([
    resnet_base,
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 classes: Normal, Pneumonia, Tuberculosis, Covid-19
])

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
from tensorflow.keras import models, layers

train_model = model.fit(
    train_data,
    validation_data=validation_data,
    epochs=50,
)

Epoch 1/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 2066s 10s/step - accuracy: 0.8196 - loss: 1.7371 - val_accuracy: 0.8421 - val_loss: 0.6087
Epoch 2/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 1681s 8s/step - accuracy: 0.9411 - loss: 0.3419 - val_accuracy: 0.8421 - val_loss: 0.2452
Epoch 3/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 1722s 9s/step - accuracy: 0.9504 - loss: 0.2241 - val_accuracy: 0.8684 - val_loss: 0.2666
Epoch 4/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 1693s 9s/step - accuracy: 0.9729 - loss: 0.1048 - val_accuracy: 0.9474 - val_loss: 0.0926
Epoch 5/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 1700s 9s/step - accuracy: 0.9753 - loss: 0.0893 - val_accuracy: 0.9474 - val_loss: 0.1396
Epoch 6/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 1694s 8s/step - accuracy: 0.9778 - loss: 0.0832 - val_accuracy: 0.9737 - val_loss: 0.0502
Epoch 7/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 1712s 9s/step - accuracy: 0.9765 - loss: 0.0891 - val_accuracy: 0.8684 - val_loss: 0.3596
Epoch 8/50
198/198 ━━━━━━━━━━━━━━━━━━━━ 1698s 9s/step - accuracy: 0.9837 - loss: 0.0630 -

In [ ]:
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.2f}")

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(image_path, model):
  img = image.load_img(img_path, target_size=(224, 224))
  img_array = image.img_to_array(img) / 255.0
  img_array = np.expand_dims(img_array, axis=0)

  predictions = model.predict(img_array)
  class_names = ["Normal", "Pneumonia", "Covid-19", "Tuberculosis"]

  predicted_label = class_names[np.argmax(predictions)]
  confidence = np.max(predictions)

  return predicted_label, confidence

#prictions
img_path = "/content/drive/MyDrive/ML chest xray data/TEST/PNEUMONIA/person101_bacteria_483.jpeg"
label, conf = predict_image(img_path, model)
print(f"Predicted: {label}, Confidence: {conf:.2f}")